# Get Nightlight Data from Google Earth Engine using GEEMap

In [1]:
import ee
import geemap

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [2]:
ee.Initialize()

## Create a GEEMap Object

In [16]:
m = geemap.Map(
    center=[-5, 15], 
    zoom=3, 
    basemap = 'Esri.WorldImagery',
    height = 1200
)

## Add Layers to the Map

In [4]:
# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2022-01-01', '2024-01-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
m.addLayer(image_night, nighttimeVis, 'Nighttime', False)


In [5]:
# add WorldPop population density layer
# https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#description
dataset_pop = ee.ImageCollection('WorldPop/GP/100m/pop') \
                  .filter(ee.Filter.date('2020-01-01', '2020-12-31'))
pop = dataset_pop.select('population')
image_pop = pop.median()
popVis = {
    'min': 0.0, 
    'max': 20.0,
    'palette': ['24126c', '1fff4f', 'd4ff50'],
    'opacity': 0.5
}
# m.addLayer(image_pop, popVis, 'Population', False)


In [17]:
# overlay country boundaries with white borders
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
style = {'color': 'ffffffff', 'width': 2, 'lineType': 'solid', 'opacity': 1}
# m.addLayer(countries, style, 'Countries', False)

sl = countries.filter(ee.Filter.eq('ADM0_NAME', 'Sierra Leone'))

# add Sierra Leone to the map
m.addLayer(sl, style, 'Sierra Leone', True)
# note: this isn't styling the countries correctly
# the "fillColor" parameter doesn't seem to work

In [18]:
# add place names
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
m.add_basemap('CartoDB.DarkMatterOnlyLabels')
# m.remove_layer('CartoDB.VoyagerOnlyLabels') # doesn't work

In [19]:
# display the map zoomed into Sierra Leone
m.setCenter(-11.779889, 8.460555, 8)

## Get the Landcover Data from Google Earth Engine

In [8]:
# pull in a global high resolution land cover dataset
# https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200
landcover = ee.ImageCollection('ESA/WorldCover/v200').first()

landcover_sl = landcover.clip(sl)

visualization = {
  'bands': ['Map'],
}

# m.addLayer(landcover_sl, visualization, 'Landcover Sierra Leone', False)

# inspect this image
print(landcover_sl.getInfo())
# inspect the bands of landcover_sl
print(landcover_sl.bandNames().getInfo())
# inspect the values of the band 'Map'
print(landcover_sl.select('Map').getInfo())



{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [4320000, 1728000], 'crs': 'EPSG:4326', 'crs_transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}], 'version': 1699537784392512, 'id': 'ESA/WorldCover/v200/2021', 'properties': {'Map_class_names': ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare / sparse vegetation', 'Snow and ice', 'Permanent water bodies', 'Herbaceous wetland', 'Mangroves', 'Moss and lichen'], 'system:time_start': 1609455600000, 'system:time_end': 1640991600000, 'Map_class_palette': ['006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0'], 'Map_class_values': [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100], 'system:asset_size': 109661138988, 'system:index': '2021'}}
['Map']
{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255

## Get Points from the Ocean off the Coast

In [20]:
m

Map(center=[8.460555, -11.779889], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [21]:
# # export the polygon just drawn on the map "m" above to a ee.eature
# ocean_sl_feat = m.draw_last_feature

# # export ee.feature to geojson file
# geemap.ee_to_geojson(ocean_sl_feat, 'data/dark_sl/ocean_sl.geojson')

# read in ocean polygon
ocean_sl = geemap.geojson_to_ee('data/dark_sl/ocean_sl.geojson')
m.addLayer(ocean_sl, {}, 'Ocean SL', False)

## Create Masks for Rural and Built-up Areas

In [16]:

# create a mask for the landcover
landcover_sl_built_mask = landcover_sl.eq(50)
landcover_sl_built = landcover_sl.updateMask(landcover_sl_built_mask)
# m.addLayer(landcover_sl_built, {'palette': 'red'}, 'Built')

# create a 10km buffer around built areas
built_buffer = landcover_sl_built.focal_max(10000, 'circle', 'meters')
# m.addLayer(built_buffer, {'palette': 'red'}, 'Built buffer', False)

# create a mask for the built_buffer
# need to unmask it to convert areas outside of mask from nodata to 0
rural_mask = built_buffer.eq(50).unmask(0).eq(0)
# select everywhere in Sierra Leone outside of the built buffer
landcover_sl_rural = landcover_sl.updateMask(rural_mask)
# m.addLayer(landcover_sl_rural, {'palette': 'brown'},  'Rural landcover', False)



In [11]:
# # select just treed areas and water areas from landcover_sl_rural
# rural_trees_mask = landcover_sl_rural.eq(10)
# rural_water_mask = landcover_sl_rural.eq(80)

# rural_trees = landcover_sl_rural.updateMask(rural_trees_mask)
# rural_water = landcover_sl_rural.updateMask(rural_water_mask)

# # add the rural tree and water layers
# m.addLayer(rural_trees, {'palette': 'green'}, 'Rural tree cover', False)
# m.addLayer(rural_water, {'palette': 'blue'}, 'Rural water', False)

## Sample These Areas

### Rural

In [36]:
# sample 50 random points from the rural areas
# note this drops any points that have been masked
rural_pts_fc = landcover_sl_rural.sample(
    region=sl,
    scale=1000, # 1000m
    numPixels=65, # want 40-47 points
    seed=44,
    projection='EPSG:4326',
    dropNulls=True, # drop any points that have been masked
    geometries=True
)

rural_pts_fc = rural_pts_fc.map(lambda f: f.set('type', 'rural'))

m.addLayer(
    rural_pts_fc, 
    {'color': '00ff00', 'pointSize': 10}, 
    'Rural tree points', 
    True
)

# export rural_pts_fc to a geojson file
geemap.ee_to_geojson(rural_pts_fc, 'data/dark_sl/rural_pts.geojson')

rural_pts_fc

In [22]:
# read in rural points
rural_pts_fc = geemap.geojson_to_ee('data/dark_sl/rural_pts.geojson')

# add to map
m.addLayer(
    rural_pts_fc, 
    {'color': '00ff00', 'pointSize': 10}, 
    'Rural points', 
    True
)

rural_pts_fc

### Built-up

In [35]:
# sample 50 random points from the built area
# note this drops any points that have been masked
built_pts_fc = landcover_sl_built.sample(
    region=sl,
    scale=1000, # 100m
    numPixels=9800, # want 40-47 points
    seed=44,
    projection='EPSG:4326',
    dropNulls=True, # drop any points that have been masked
    geometries=True
)

built_pts_fc = built_pts_fc.map(lambda f: f.set('type', 'built'))

# drop the geodesic property in geometry
built_pts_fc = built_pts_fc.map(lambda f: f.setGeometry(f.geometry().centroid()))

m.addLayer(
    built_pts_fc, 
    {'color': 'ff0000', 'pointSize': 10}, 
    'built points', 
    True
)

# export built_pts_fc to a geojson file
geemap.ee_to_geojson(built_pts_fc, 'data/dark_sl/built_pts.geojson')

built_pts_fc

In [23]:
# read in built points
built_pts_fc = geemap.geojson_to_ee('data/dark_sl/built_pts.geojson')

# add to map
m.addLayer(
    built_pts_fc, 
    {'color': 'ff0000', 'pointSize': 10}, 
    'built points', 
    True
)

built_pts_fc

### Ocean

In [37]:
# sample 50 random points from the ocean
# read in ocean_sl_feat from geojson file
ocean_sl_feat = geemap.geojson_to_ee('data/dark_sl/ocean_sl.geojson')

# note this drops any points that have been masked
ocean_pts_fc = landcover.sample(
    region=ocean_sl_feat,
    scale=1000, # 100m
    numPixels=47, # want 40 points
    seed=44,
    projection='EPSG:4326',
    dropNulls=False, 
    geometries=True
)

ocean_pts_fc = ocean_pts_fc.map(lambda f: f.set('type', 'ocean'))

m.addLayer(
    ocean_pts_fc, 
    {'color': 'ff0000', 'pointSize': 10}, 
    'ocean points', 
    True
)

# export ocean_pts_fc to a geojson file
geemap.ee_to_geojson(ocean_pts_fc, 'data/dark_sl/ocean_pts.geojson')

ocean_pts_fc

In [24]:
# read in built points
ocean_pts_fc = geemap.geojson_to_ee('data/dark_sl/ocean_pts.geojson')

# add to map
m.addLayer(
    ocean_pts_fc, 
    {'color': '0000ff', 'pointSize': 10}, 
    'ocean points', 
    True
)

ocean_pts_fc

## Create a 1km Buffer Around the Sampled Points

In [38]:
# read in the geojson files
rural_pts_fc = geemap.geojson_to_ee('data/dark_sl/rural_pts.geojson')
built_pts_fc = geemap.geojson_to_ee('data/dark_sl/built_pts.geojson')
ocean_pts_fc = geemap.geojson_to_ee('data/dark_sl/ocean_pts.geojson')

In [48]:
# merge into a single fc
sl_pts_fc = rural_pts_fc.merge(built_pts_fc).merge(ocean_pts_fc)

In [49]:
# create a 1km buffer around the points
sl_pts_buffer = sl_pts_fc.map(lambda f: f.buffer(1000))


## Get the Nightlights Values for each buffer

In [50]:
# get the image collectino of nightlight images
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2014-01-01', '2024-01-01'))
# select the avg_rad band
nighttime_ic = dataset_night.select('avg_rad')
# stack the images into a single image
nighttime_ic_stack = nighttime_ic.toBands()
nighttime_ic_stack

In [51]:
# apply the reduceRegions for each of the point buffers
sl_fc = nighttime_ic_stack.reduceRegions(
    collection=sl_pts_buffer, 
    reducer=ee.Reducer.mean(), 
    scale=50
)

# inspect one
sl_fc


In [52]:
# change the geometry to the center of the buffer
sl_fc = sl_fc.map(lambda f: f.setGeometry(f.geometry().centroid()))

## Convert the Feature Collections to GeoDataFrames and Clean Data

In [53]:
# convert fc to gdf
sl_gdf = geemap.ee_to_geopandas(sl_fc)

# remove "_avg_rad" from the column names that contain it
sl_gdf.columns = sl_gdf.columns.str.replace('_avg_rad', '')
    
# export geodataframe to geojson
sl_gdf.to_file('data/dark_sl/sl_nightlights.geojson', driver='GeoJSON', index=False)
